In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

In [5]:
# 넘파이 데이터 불러오기
X_train, X_test, Y_train, Y_test = np.load(
    '/content/datasets/news_data_max_27_size24151.npy',
    allow_pickle=True)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(24367, 27)
(2708, 27)
(24367, 6)
(2708, 6)


In [6]:
model = Sequential() 
model.add(Embedding(24151, 300, input_length=27))
# Embedding : 벡터라이징을 해주는 함수, input_length : 문장은 길이만 있기에 length
model.add(Conv1D(32, kernel_size=5,
                 padding='same', activation='relu')) # 문장의 관계 Conv1D
model.add(MaxPool1D(pool_size=1))
# 순서가 있는 데이터, LSTM 사용
model.add(LSTM(128, activation='tanh',
               return_sequences = True))
model.add(Dropout(0.3))
model.add(LSTM(64, activation='tanh',
               return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(64, activation='tanh'))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(6, activation='softmax')) # 다중분류기 softmax를 사용하여 확률값으로
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 27, 300)           7245300   
_________________________________________________________________
conv1d (Conv1D)              (None, 27, 32)            48032     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 27, 32)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 27, 128)           82432     
_________________________________________________________________
dropout (Dropout)            (None, 27, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 27, 64)            49408     
_________________________________________________________________
dropout_1 (Dropout)          (None, 27, 64)            0

In [7]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])
fit_hist = model.fit(X_train, Y_train, batch_size=50,
                     epochs=6, validation_data=(X_test, Y_test))

Epoch 1/6
488/488 [==============================] - 75s 81ms/step - loss: 1.2109 - accuracy: 0.5255 - val_loss: 0.8620 - val_accuracy: 0.7260
Epoch 2/6
488/488 [==============================] - 38s 78ms/step - loss: 0.6261 - accuracy: 0.7952 - val_loss: 0.7505 - val_accuracy: 0.7570
Epoch 3/6
488/488 [==============================] - 38s 79ms/step - loss: 0.3704 - accuracy: 0.8866 - val_loss: 0.8182 - val_accuracy: 0.7500
Epoch 4/6
488/488 [==============================] - 38s 79ms/step - loss: 0.2411 - accuracy: 0.9272 - val_loss: 0.8502 - val_accuracy: 0.7496
Epoch 5/6
488/488 [==============================] - 38s 78ms/step - loss: 0.1653 - accuracy: 0.9524 - val_loss: 0.9787 - val_accuracy: 0.7459
Epoch 6/6
488/488 [==============================] - 38s 78ms/step - loss: 0.1195 - accuracy: 0.9655 - val_loss: 1.1029 - val_accuracy: 0.7400


In [8]:
score = model.evaluate(X_test, Y_test)
print(score[1])

85/85 [==============================] - 0s 5ms/step - loss: 1.1029 - accuracy: 0.7400
0.740029513835907


In [9]:
model.save('/content/models/news_classification_{}.h5'.format(score[1]))